In [1]:
# %pip install -U lightgbm==3.3.2

In [2]:
# %pip install implicit

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
%pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 37.1 MB/s  0:00:006m0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [gensim]2m2/3 [gensim]
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np

import pickle
from tqdm import tqdm
import gc
from pathlib import Path

from gensim.models import Word2Vec

import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell

warnings.filterwarnings("ignore")
InteractiveShell.ast_node_interactivity = "all"
tqdm.pandas()

In [6]:
from src.data import DataHelper
import pickle

In [7]:
from collections import defaultdict

In [8]:
data_dir = Path("data")
model_dir = Path("models")

In [9]:
%pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import gc
from pathlib import Path
data_dir = Path("data")
model_dir = Path("models")

TRAIN_WEEK_NUM = 4
WEEK_NUM = TRAIN_WEEK_NUM + 2

VERSION_NAME = "Recall 1"
TEST = True # * Set as `False` when do local experiments to save time
# Ensure base data dir exists, then create interim/processed subfolders

data_dir.mkdir(parents=True, exist_ok=True)
model_dir.mkdir(parents=True, exist_ok=True)
(data_dir / "interim" / VERSION_NAME).mkdir(parents=True, exist_ok=True)
(data_dir / "index_id_map" / VERSION_NAME).mkdir(parents=True, exist_ok=True)
(data_dir / "external" / VERSION_NAME).mkdir(parents=True, exist_ok=True)

In [10]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thangndk67/h-and-m-csv-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/codespace/.cache/kagglehub/datasets/thangndk67/h-and-m-csv-dataset/versions/5


In [11]:
dh = DataHelper(data_dir)
# data = dh.preprocess_data(save=True, name="encoded_full") # * run only once, processed data will be saved
data = dh.load_data(name="encoded_full",load_path= path)

### Train


In [12]:
inter = data['inter']

In [13]:
inter['t_dat'] = pd.to_datetime(inter['t_dat'])
last_week_start = pd.to_datetime("2020-08-19")
inter = inter.loc[(inter.t_dat < last_week_start)]

In [14]:
feedid_seq_list = inter.groupby(['customer_id']).article_id.apply(lambda x: [str(id) for id in x] ).values

In [17]:
# model_sg = Word2Vec(feedid_seq_list,  size=128, window=32, min_count=1, sg=0, sample=1e-3, negative=15, workers=32, seed=1, iter=10)
# model_sg.save(open(model_dir/'articleid_model_cbow.model','wb'))
model_sg = Word2Vec(feedid_seq_list,  vector_size=128, window=32, min_count=1, sg=1, sample=1e-3, negative=15, workers=32, seed=1)
model_sg.save(open(model_dir/'articleid_model_skipgram.model','wb'))

KeyboardInterrupt: 

In [ ]:
# model_sg = pickle.load(open(model_dir/'articleid_model_cbow.model','rb'))
model_sg = pickle.load(open(model_dir/'articleid_model_skipgram.model','rb'))

In [ ]:
# * Customer Embedding
feedid_seq_list = inter.groupby(['customer_id']).article_id.apply(lambda x: [str(id) for id in x] ).reset_index()

full_users = data['user']['customer_id'].values
customer_embedding = np.ones((len(full_users)+1, 128))/128

for uid, items in tqdm(feedid_seq_list.values):
    if len(items)>1:
        vec = np.mean(list(map(lambda x: model_sg[x], items)), axis=0)
    else:
        vec = model_sg[items[0]]
    customer_embedding[uid] = vec/np.sqrt(np.sum(vec**2))

100%|██████████| 1334713/1334713 [04:02<00:00, 5512.67it/s]


In [ ]:
del full_users, feedid_seq_list, inter
gc.collect()

53

In [ ]:
# customer_embedding.dump(data_dir/'external'/'w2v_user_embd.npy')
customer_embedding.dump(data_dir/'external'/'w2v_skipgram_user_embd.npy')

In [ ]:
# * Article Embedding
full_items = data['item']['article_id'].values
article_embedding = np.ones((len(full_items)+1, 128))/128
for item in tqdm(full_items):
    try:
        vec = model_sg[str(item)]
    except:
        vec = article_embedding[item]
    article_embedding[item,:] = vec/np.sqrt(np.sum(vec**2))

100%|██████████| 105542/105542 [00:03<00:00, 34165.45it/s]


In [ ]:
# article_embedding.dump(data_dir/'external'/'w2v_item_embd.npy')
article_embedding.dump(data_dir/'external'/'w2v_skipgram_item_embd.npy')

In [ ]:
# * Product_code Embedding
full_products = list(data['item']['product_code'].unique())

In [ ]:
product_embd_dict = {}
for pid, items in tqdm(data['item'].groupby('product_code')):
    embd = article_embedding[items['article_id'].values]
    embd = np.sum(embd, axis=0)
    product_embd_dict[pid] = embd/np.sqrt(np.sum(embd**2))

100%|██████████| 47224/47224 [00:07<00:00, 5918.84it/s]


In [ ]:
product_embedding = np.ones((len(full_products)+1, 128))/128
for pid,embd in tqdm(product_embd_dict.items()):
    product_embedding[pid,:] = embd

100%|██████████| 47224/47224 [00:00<00:00, 667652.54it/s]


In [ ]:
# product_embedding.dump(data_dir/'external'/'w2v_product_embd.npy')
product_embedding.dump(data_dir/'external'/'w2v_skipgram_product_embd.npy')

---

In [ ]:
# * Product_code Embedding
full_products = list(data['item']['product_code'].unique())

In [ ]:
dssm_item_embd = np.load(data_dir/'external'/'dssm_item_embd.npy', allow_pickle=True)

product_embd_dict = {}
for pid, items in tqdm(data['item'].groupby('product_code')):
    embd = dssm_item_embd[items['article_id'].values-1]
    embd = np.sum(embd, axis=0)
    product_embd_dict[pid] = embd/np.sqrt(np.sum(embd**2))

product_embedding = np.ones((len(full_products), 128))/128
for pid,embd in tqdm(product_embd_dict.items()):
    product_embedding[pid-1,:] = embd

In [ ]:
product_embedding.dump(data_dir/'external'/'dssm_product_embd.npy')

In [ ]:
yt_item_embd = np.load(data_dir/'external'/'yt_item_embd.npy', allow_pickle=True)

product_embd_dict = {}
for pid, items in tqdm(data['item'].groupby('product_code')):
    embd = yt_item_embd[items['article_id'].values-1]
    embd = np.sum(embd, axis=0)
    product_embd_dict[pid] = embd/np.sqrt(np.sum(embd**2))

product_embedding = np.ones((len(full_products), 128))/128
for pid,embd in tqdm(product_embd_dict.items()):
    product_embedding[pid-1,:] = embd

100%|██████████| 47224/47224 [00:00<00:00, 859955.42it/s]


In [ ]:
product_embedding.dump(data_dir/'external'/'yt_product_embd.npy')